In [1]:
import json
import pandas as pd

import matplotlib as mtl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import numpy as np
from folium import plugins
from folium.plugins import HeatMap


# Yelp Dataset (https://www.kaggle.com/yelp-dataset/yelp-dataset)

In [2]:
#Replace the file location here if you are running it on your machine
N = 10000
with open('D:/Machine Learning/YelpFull/yelp_academic_dataset_checkin.json', 'rb') as datafile:
    dataSubset = [next(datafile) for x in range(N)]

In [3]:
processedData = "[" + ','.join([d.strip().decode('UTF-8') for d in dataSubset]) + "]"
dfCheckin = pd.read_json(processedData)

In [4]:
#Subset the number of lines (e.g. reading first 10,000 lines here)
N = 10000
#Replace the file location here if you are running it on your machine
with open('D:/Machine Learning/YelpFull/yelp_academic_dataset_business.json', 'rb') as datafile:
    dataSubset = [next(datafile) for x in range(N)]

## Let's look at a sample data item

In [5]:
parsed = json.loads(dataSubset[0]) #[0] gives the first element
print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "address": "2818 E Camino Acequia Drive",
    "attributes": {
        "GoodForKids": "False"
    },
    "business_id": "1SWheh84yJXfytovILXOAQ",
    "categories": "Golf, Active Life",
    "city": "Phoenix",
    "hours": null,
    "is_open": 0,
    "latitude": 33.5221425,
    "longitude": -112.0184807,
    "name": "Arizona Biltmore Golf Club",
    "postal_code": "85016",
    "review_count": 5,
    "stars": 3.0,
    "state": "AZ"
}


In [6]:
processedData = "[" + ','.join([d.strip().decode('UTF-8') for d in dataSubset]) + "]"
# now, load it into pandas (N number of rows)
dfData = pd.read_json(processedData)
dfLocs = pd.merge(dfCheckin,dfData,on='business_id')

## Calculating the hourly check-ins for each business

In [7]:
activity=[[] for _ in range(24)]
for index,rows in dfLocs.iterrows():
    dates = rows['date']
    time = str(dates).split(', ')
    for i in time:
        i=str(i).split(' ')
        hour=str(i[1]).split(':')
        activity[int(hour[0])].append([rows['latitude'],rows['longitude']])

## Heatmap showing hourly activity for every business

In [8]:
lat = 43.860397
lon = -79.303184
zoom_start=9
m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

hm = plugins.HeatMapWithTime(activity,max_opacity=0.3,auto_play=True,display_index=True)
hm.add_to(m)
m